In [ ]:
using Plots, LightGraphs, SparseArrays, SimpleWeightedGraphs
using Statistics, BenchmarkTools, LinearAlgebra, ProgressMeter
using Distributions, Base.Threads
using Base.GC
using JuMP, ECOS, CSDP, Ipopt, SDPA, ProxSDP, OSQP, SCS, COSMO
plotly();

# Helpers

In [ ]:
function flipcoin(p)
    x = rand()
    if x <= p
        return 1
    else
        return 0
    end
end

# Type 1 graphs
function genGraph(n,p)
    G = randn(n,n)
    for i = 1:n
        G[i,i] = 0
        for j = 1:i-1
            if flipcoin(p) == 0
                G[i,j] = 0
                G[j,i] = 0
            else
                G[i,j] = G[j,i] 
            end
        end
    end
    
    return G
end  

function Labels(A,k)
    return Set(partialsortperm(A, 1:k, rev=true))
end

# Type 2 graphs
function genGraphDir(n,p,alpha,k)
    d = Dirichlet(alpha)
    L = Array{Set{Int},1}(undef,n)
    for i = 1:n
        L[i] = Labels(rand(d),k)
    end
    
    G = zeros(n,n)
    for i = 1:n
        G[i,i] = 0
        for j = 1:i-1
            if flipcoin(p) == 0
                G[i,j] = 0
                G[j,i] = 0
            else
                if isempty(intersect(L[i],L[j]))
                    G[i,j] = 1
                    G[j,i] = 1
                else
                    G[i,j] = 0.00000001
                    G[j,i] = 0.00000001
                end
            end
        end
    end
    
    return G,L
end  

function enumerate_paths2(s)
    P = Array{Any,1}(undef,size(s.parents, 1))
    for v = 1:size(s.parents, 1)
        P[v] = LightGraphs.enumerate_paths(s, v)
    end
    
    return P
end

function calculateSTDEV(x,y)
    error = 0.0
    for i=1:length(y)
        error = error + (x-y[i])^2
    end
    
    return sqrt(error/length(y))
end

# Running experiment for standard solvers. 

Modify the line set_optimizer(model,*.optimizer) where * is the name of the optimizer. For the commerical solvers you must have a liscence and those solvers independetly installed

In [ ]:
T = zeros(50)
for s = 1:3
    n = 100*s
    time = zeros(5)
    for r = 1:5 
        G = 0
        model = 0
        D = 0
        G = genGraph(n,1.1)
        
        t = @elapsed model = Model()
        t += @elapsed @variable(model, D[1:n,1:n]);
        t += @elapsed @objective(model, Min, sum((D-G).^2));
        t += @elapsed for i = 1:n
            for j = 1:n
                @constraint(model, D[i,j] == D[j,i])
            end
        end
                
        t += @elapsed for i = 1:n
            for j = 1:n
                for k = 1:n
                    @constraint(model, D[i,j] <= D[i,k]+D[k,j])
                end
            end
        end
        
        t += @elapsed for i = 1:n
            @constraint(model, D[i,i] ==  0)
        end
        
        t += @elapsed set_optimizer(model, CSDP.Optimizer)
        t += @elapsed optimize!(model) 
        print(termination_status(model))
        time[r] = solve_time(model)
        @show(t)
        flush(stdout)
    end
    T[s] = mean(time)
    @show(T[s])
    flush(stdout)
end

In [ ]:
T

# Our implementation of the Brickell et al. algorithm 

In [ ]:
function Bregman3Cycle(D)
    (n,n) = size(D)
    Z = zeros(n,n,n)
    maxD = 2
    count = 0
    while(maxD > 1e-10)
        maxD = 0
        for i = 1:n
            for j = 1:i-1
                for k = 1:j-1
                    d = (-1*D[i,k] + D[i,j] + D[j,k])/3 
                    c = min(d,Z[i,j,k])
                    Z[i,j,k] -= c
                    D[i,k] += c
                    D[k,i] += c
                    D[i,j] -= c
                    D[j,i] -= c
                    D[j,k] -= c
                    D[k,j] -= c
                    d = (-1*D[j,i] + D[j,k] + D[k,i])/3 
                    c = min(d,Z[j,k,i])
                    Z[j,k,i] -= c
                    D[i,k] -= c
                    D[k,i] -= c
                    D[i,j] += c
                    D[j,i] += c
                    D[j,k] -= c
                    D[k,j] -= c
                    d = (-1*D[k,j] + D[k,i] + D[i,j])/3 
                    c = min(d,Z[k,i,j])
                    Z[k,i,j] -= c
                    D[i,k] -= c
                    D[k,i] -= c
                    D[i,j] -= c
                    D[j,i] -= c
                    D[j,k] += c
                    D[k,j] += c
                end
            end
        end 
        FS =  LightGraphs.floyd_warshall_shortest_paths(SimpleWeightedGraph(D))
        maxD = norm(FS.dists-D)
        count += 1 
    end
    return (D,count)
end 

# Our algorithm

In [ ]:
function BregmanOrig(D)
    (n,n) = size(D)
    #G = sparse(copy(D))
    g = SimpleWeightedGraph(D)
    Z = Dict()
    Z′ = spzeros(n,n)
    maxD = 2
    count = 0
    while(maxD > 1e-10)
        
        for p in keys(Z)
            z = Z[p]
            l = length(p)
            u = p[1]
            v = p[l]
            d = -1*g.weights[u,v]
            for i = 1:l-1
                d = d + g.weights[p[i], p[i+1]]
            end
            c = min(d/l,z)
            for i = 1:l-1
                g.weights[p[i],p[i+1]] -= c
                g.weights[p[i+1],p[i]] -= c
            end
            g.weights[u,v] += c
            g.weights[v,u] += c
            if z == c
                delete!(Z,p)
            else
                Z[p] -= c
            end
        end
    
        for i = 1:n
            for j = 1:i-1
                c = min(g.weights[j,i] - 1e-14,Z′[j,i])
                g.weights[j,i] -= c
                g.weights[i,j] -= c
                Z′[j,i] -= c
                Z′[i,j] -= c             
            end
        end
        
        FS = LightGraphs.floyd_warshall_shortest_paths(g)
        U = FS.dists
        P = enumerate_paths2(FS)
        maxD = 0
        
        for i = 1:n
            for j = 1:i-1
                if g.weights[j,i] - U[j,i] > 0  
                    p = P[j][i] #enumerate_paths2(FS,i,j)
                    l = length(p)
                    u = p[1]
                    v = p[l]
                    d = -1*g.weights[u,v]
                    for k = 1:l-1
                        d = d + g.weights[p[k], p[k+1]]
                    end
                    if d < 0
                        c=d/l
                        for k = 1:l-1
                            g.weights[p[k],p[k+1]] -= c
                            g.weights[p[k+1],p[k]] -= c
                        end
                        g.weights[p[1],p[l]] += c
                        g.weights[p[l],p[1]] += c
                        if haskey(Z,p)
                            Z[p] = Z[p] - c
                        else
                            Z[p] = -1*c
                        end
                        if abs(d) > maxD
                            maxD = abs(d)
                        end
                    end
                end   
            end
        end
        FS =  LightGraphs.floyd_warshall_shortest_paths(g)
        maxD = norm(FS.dists-g.weights)
        
        count+=1
    end
    return length(Z)
end

# The code to run the metric nearness experiemnt

In [ ]:
T = zeros(50);
Z = zeros(50);
errorbars = zeros(50,1);
errorbarsZ = zeros(50,1);
T2 = zeros(50);
errorbars2 = zeros(50,1);

In [ ]:
for i = 1:16
    time = zeros(5)
    time2 = zeros(5)
    z = zeros(5)
    n = 100*i
    for j = 1:5 
        G = genGraph(100*i,1.1)
        #G,L = genGraphDir(100*i,1.1, ones(100),20)
        time[j] = @elapsed z[j] = BregmanOrig(copy(G))
        @show(time[j])
        time2[j] = @elapsed Bregman3Cycle(copy(G))
        @show(time2[j])
    end
    errorbarsZ[i] = calculateSTDEV(Z[i],z)
    T[i] = mean(time)
    T2[i] = mean(time2)
    errorbars[i] = calculateSTDEV(T[i],time)
    errorbars2[i] = calculateSTDEV(T2[i],time2)
    @show(T[i],T2[i])
end